In [3]:
# !pip install alpha_vantage

In [4]:
# !pip install pandas_datareader

In [22]:
# Import Statement
import numpy as np
import pandas as pd
import datetime as date
import requests
from pandas_datareader import data
from sklearn.neighbors import KNeighborsRegressor

# from alpha_vantage.cryptocurrencies import CryptoCurrencies
API_KEY = 'VQTPJ1YBJF3UBLZB'
# app = CryptoCurrencies(API_KEY) 

pd.options.display.float_format = '{:,.2f}'.format

In [25]:
# by using ALPHA VANTAGE API
# https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=CNY&apikey=demo&datatype=csv

class Crypto_prediction:
  def __init__(self,symbol):
    self.df = None
    self.symbol = symbol

  def get_df(self):
    base_url = 'https://www.alphavantage.co/query?'
    params = {'function': 'DIGITAL_CURRENCY_DAILY',
              'symbol': self.symbol,
              'market': 'INR',
              'apikey': API_KEY
              }

    response_dict = requests.get(base_url, params=params).json()

    key_ar = list(response_dict.keys())
    if len(key_ar) == 1:
      print(key_ar)
      self.df =  pd.DataFrame(np.nan,index=[0,1],columns=['A'])
      return self.df
    #Convert to pandas dataframe

    df = pd.DataFrame.from_dict(response_dict[key_ar[-1]], orient='index')

    # 1
    for each in df.columns:
      df[each]= df[each].astype(float)

    # 2
    df = df[::-1]
    self.df = df
    return self.df


  def get_prediction(self):
    df = self.get_df()
    
    if df.isna().any().any():
      return "Time limit exceeded, Alpha Vantage's standard API call frequency is 5 calls per minute and 500 calls per day"
    

    cls_point = df.loc[:, ['4a. close (INR)']]
    open_point = df.loc[:, ['1a. open (INR)']]
    high_point = df.loc[:, ['2a. high (INR)']]
    low_point = df.loc[:, ['3a. low (INR)']]
    vol = df.loc[:, ['5. volume']]

    feature = []
    cls_point_arr = []
    target_label = []
# creating feature set
    for i in range((len(cls_point.index) - 1)):
      feature.append([open_point.iloc[i].values[0],
                      high_point.iloc[i].values[0],
                      low_point.iloc[i].values[0],
                      vol.iloc[i].values[0]]
                      )
# creating  set        
    for i in range(len(cls_point.index)):
      cls_point_arr.append(cls_point.iloc[i].values[0])
      
      close_point_shifted_right = [cls_point_arr[i] for i in range(1, len(cls_point_arr))]

# target label : last value of dataset
      j = len(cls_point.index) - 1
      target_label.append([open_point.iloc[j].values[0],
                          high_point.iloc[j].values[0],
                          low_point.iloc[j].values[0],
                          vol.iloc[j].values[0]]
                        )

    knnregressor = KNeighborsRegressor(n_neighbors=31, weights='uniform',algorithm='kd_tree')
    knnregressor.fit(feature, close_point_shifted_right)
    val = knnregressor.predict(target_label)

    return (close_point_shifted_right[-1],val[0])
  # 26,96,232.08

In [26]:
"""
Bitcoin.
Ethereum.
Ripple XRP.
Litecoin.
NEO.
IOTA.
Cardano (ADA)
Polkadot (DOT)
Stellar (XLM)
Tether (USDT)
"""
arr = ['BTC','ETC','XRP','LTC','NEO','DICE']

for i in arr:
  obj = Crypto_prediction(i)
  obj.get_df()
  print(i+": ",obj.get_prediction())


BTC:  (2719430.7186, 2578903.443654838)
ETC:  (568.1681475, 480.59142331451613)
['Note']
XRP:  Time limit exceeded, Alpha Vantage's standard API call frequency is 5 calls per minute and 500 calls per day
['Note']
['Note']
LTC:  Time limit exceeded, Alpha Vantage's standard API call frequency is 5 calls per minute and 500 calls per day
['Note']
['Note']
NEO:  Time limit exceeded, Alpha Vantage's standard API call frequency is 5 calls per minute and 500 calls per day
['Note']
['Note']
DICE:  Time limit exceeded, Alpha Vantage's standard API call frequency is 5 calls per minute and 500 calls per day


In [ ]:
# !pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask

app = Flask(__name__)
# run_with_ngrok(app)

@app.route('/')
def index():
  """
  Bitcoin.
  Ethereum.
  Ripple XRP.
  Litecoin.
  NEO.
  IOTA.
  Cardano (ADA)
  Polkadot (DOT)
  Stellar (XLM)
  Tether (USDT)
  """
  arr = ['BTC','ETC','XRP','LTC','NEO','IOTA','ADA','DOT','XLM','USDT']
  for i in arr:
    obj = Crypto_prediction(i)
    preds = obj.get_prediction()

  return f"<h1> {preds} </h1>"

app.run()

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score

# def_buy_or_sell(df):
#   df['Open-Close']= df['1a. open (INR)'] - df['3a. low (INR)']
#   df['High-Low']  = df['2a. high (INR)'] - df['4a. close (INR)']
#   df =df.dropna()
#   X= df[['Open-Close', 'High-Low']]
#   X.head()

#   # Target variable
#   Y= np.where(df['4a. close (INR)'].shift(-1)>df['4a. close (INR)'],1,-1)

#   split_percentage = 0.7
#   split = int(split_percentage*len(df))

#   X_train = X[:split]
#   Y_train = Y[:split]

#   X_test = X[split:]
#   Y_test = Y[split:]

#   knn = KNeighborsClassifier(n_neighbors=15)

#   # fit the model
#   knn.fit(X_train, Y_train)

#   # Accuracy Score
#   accuracy_train = accuracy_score(Y_train, knn.predict(X_train))
#   accuracy_test = accuracy_score(Y_test, knn.predict(X_test))

#   print ('Train_data Accuracy: %.2f' %accuracy_train)
#   print ('Test_data Accuracy: %.2f' %accuracy_test)